In [1]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00


In [2]:
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.0 MB/s eta 0:00:00


The content of individual GitHub issues may be longer than what an embedding model can take as input. If we want to embed all of the available content, we need to chunk the documents into appropriately sized pieces.

The most common and straightforward approach to chunking is to define a fixed size of chunks and whether there should be any overlap between them. Keeping some overlap between chunks allows us to preserve some semantic context between the chunks. The recommended splitter for generic text is the [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), and that's what we'll use here.

In [6]:
import os
from langchain_community.document_loaders.csv_loader import CSVLoader

In [7]:
documents=[]
csv_folder_path="/content/sample_data/test_folder"
for file in os.listdir(csv_folder_path):
        if file.endswith('.csv'):
            csv_path = os.path.join(csv_folder_path, file)
            loader = CSVLoader(csv_path)
            documents.extend(loader.load())

In [11]:
documents[100]

Document(page_content='Dist Code: 12\nYear: 2011\nState Code: 6\nState Name: Madhya Pradesh\nDist Name: Mandla\nRICE AREA (1000 ha): 198.7\nRICE PRODUCTION (1000 tons): 201.8\nRICE YIELD (Kg per ha): 1015.6\nWHEAT AREA (1000 ha): 68\nWHEAT PRODUCTION (1000 tons): 51.3\nWHEAT YIELD (Kg per ha): 754.41\nKHARIF SORGHUM AREA (1000 ha): 0.14\nKHARIF SORGHUM PRODUCTION (1000 tons): 0.12\nKHARIF SORGHUM YIELD (Kg per ha): 911.76\nRABI SORGHUM AREA (1000 ha): 0\nRABI SORGHUM PRODUCTION (1000 tons): 0\nRABI SORGHUM YIELD (Kg per ha): 0\nSORGHUM AREA (1000 ha): 0.14\nSORGHUM PRODUCTION (1000 tons): 0.12\nSORGHUM YIELD (Kg per ha): 911.76\nPEARL MILLET AREA (1000 ha): 0\nPEARL MILLET PRODUCTION (1000 tons): 0\nPEARL MILLET YIELD (Kg per ha): 0\nMAIZE AREA (1000 ha): 42.1\nMAIZE PRODUCTION (1000 tons): 32.1\nMAIZE YIELD (Kg per ha): 762.47\nFINGER MILLET AREA (1000 ha): 0.5\nFINGER MILLET PRODUCTION (1000 tons): 0.1\nFINGER MILLET YIELD (Kg per ha): 200\nBARLEY AREA (1000 ha): 0\nBARLEY PRODUCTION

In [13]:
documents[0]

Document(page_content='\ufeffCompany: Pfizer\nAcquired: Seagen\nDate(dd-mm-yyyy): 13-01-2024\nValue (in billions USD): 43', metadata={'source': '/content/sample_data/test_folder/test_vectordb.csv', 'row': 0})

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(documents)

## Create the embeddings + retriever

Now that the docs are all of the appropriate size, we can create a database with their embeddings.

To create document chunk embeddings we'll use the `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5) embeddings model. There are many other embeddings models available on the Hub, and you can keep an eye on the best performing ones by checking the [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).


To create the vector database, we'll use `FAISS`, a library developed by Facebook AI. This library offers efficient similarity search and clustering of dense vectors, which is what we need here. FAISS is currently one of the most used libraries for NN search in massive datasets.

We'll access both the embeddings model and FAISS via LangChain API.

In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='thenlper/gte-large'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [16]:
indexes=db.index

In [17]:
indexes.reconstruct(0)

array([ 0.00211448, -0.00786009,  0.02203421, ..., -0.02032324,
       -0.01553673, -0.01845622], dtype=float32)

In [18]:
len(indexes.reconstruct(0))

1024

In [19]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [20]:
query="which company acquired Abceutics"

In [21]:
retriver=db.as_retriever(search_type="mmr",search_kwargs={"k":1})

In [22]:
retriver.get_relevant_documents(query)

[Document(page_content='\ufeffCompany: Merck\nAcquired: Abceutics\nDate(dd-mm-yyyy): 05-04-2024\nValue (in billions USD): 0.2', metadata={'source': '/content/sample_data/test_folder/test_vectordb.csv', 'row': 5})]

In [23]:
query="list the software company"

In [25]:
retriver.get_relevant_documents(query)

[Document(page_content='\ufeffCompany: Google\nAcquired: Kaggle\nDate(dd-mm-yyyy): undisclosed\nValue (in billions USD): undisclosed', metadata={'source': '/content/sample_data/test_folder/test_vectordb.csv', 'row': 10})]

In [26]:
query="list the pharmaceutical company"

In [27]:
retriver.get_relevant_documents(query)

[Document(page_content='\ufeffCompany: Bristol Myers Squibb\nAcquired: Mirati\nDate(dd-mm-yyyy): 15-12-2023\nValue (in billions USD): 5.8', metadata={'source': '/content/sample_data/test_folder/test_vectordb.csv', 'row': 3})]

The vector database and retriever are now set up, next we need to set up the next piece of the chain - the model.

## Load quantized model

For this example, we chose [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a small but powerful model.

With many models being released every week, you may want to substitute this model to the latest and greatest. The best way to keep track of open source LLMs is to check the [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

To make inference faster, we will load the quantized version of the model:

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## Setup the LLM chain

Finally, we have all the pieces we need to set up the LLM chain.

First, create a text_generation pipeline using the loaded model and its tokenizer.

Next, create a prompt template - this should follow the format of the model, so if you substitute the model checkpoint, make sure to use the appropriate formatting.

In [22]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
SYSTEM:
Answer the question based on your knowledge. use the below context for help
{context}
dont provide the context in answer

QUESTION:
{question}

ANSWER:

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Note: _You can also use `tokenizer.apply_chat_template` to convert a list of messages (as dicts: `{'role': 'user', 'content': '(...)'}`) into a string with the appropriate chat format._


Finally, we need to combine the `llm_chain` with the retriever to create a RAG chain. We pass the original question through to the final generation step, as well as the retrieved context docs:

In [23]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


## Compare the results

Let's see the difference RAG makes in generating answers to the library-specific questions.

In [27]:
def word_wrap(text, width):
    for paragraph in text.splitlines():
        line_len, line_buf = 0, []
        for word in paragraph.split():
            # If appending the word would cause overflow, flush the line.
            if line_len + len(line_buf) + len(word) > width:
                yield ' '.join(line_buf)
                line_len, line_buf = 0, []
            # If just this word alone would overflow, break it by force.
            while len(word) > width:
                yield word[:width]
                word = word[width:]
            line_buf.append(word)
            line_len += len(word)
        if line_buf:
            yield ' '.join(line_buf)

In [28]:
question = "which company acquired Abceutics "

First, let's see what kind of answer we can get with just the model itself, no context added:

In [34]:
for line in word_wrap(llm_chain.invoke({"context":"", "question": question}),100):
  print(line)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SYSTEM:
Answer the question based on your knowledge. use the below context for help
dont provide the context in answer
QUESTION:
which company acquired Abceutics
ANSWER:
<|assistant|>
I do not have access to real-time information or current events. Please provide the context of which
abceutics you are referring to, as there may be multiple companies with this name. Based on my
knowledge, I am not aware of any recent acquisitions involving a company named abceutics. However,
some notable acquisitions in the pharmaceutical industry include pfizer's acquisition of karl
landsteiner haus, novartis' acquisition of the covid-19 vaccine technology from moderna, and bayer's
acquisition of bluebird bio's gene therapy portfolio. Without further context, it is impossible to
determine if any of these companies acquired abceutics.


In [35]:
for line in word_wrap(rag_chain.invoke(question),100):
  print(line)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SYSTEM:
Answer the question based on your knowledge. use the below context for help
[Document(page_content='\ufeffCompany: Merck\nAcquired: Abceutics\nDate(dd-mm-yyyy):
05-04-2024\nValue (in billions USD): 0.2', metadata={'source': '/content/test_vectordb.csv', 'row':
5}), Document(page_content='\ufeffCompany: Google\nAcquired: Kaggle\nDate(dd-mm-yyyy):
undisclosed\nValue (in billions USD): undisclosed', metadata={'source':
'/content/test_vectordb.csv', 'row': 10}), Document(page_content='\ufeffCompany: Bristol Myers
Squibb\nAcquired: RayzeBio\nDate(dd-mm-yyyy): 05-01-2024\nValue (in billions USD): 4.1',
metadata={'source': '/content/test_vectordb.csv', 'row': 2}), Document(page_content='\ufeffCompany:
Bristol Myers Squibb\nAcquired: Mirati\nDate(dd-mm-yyyy): 15-12-2023\nValue (in billions USD): 5.8',
metadata={'source': '/content/test_vectordb.csv', 'row': 3})]
dont provide the context in answer
QUESTION:
which company acquired Abceutics
ANSWER:
The company that acquired Abceutics is 

In [36]:
question = "which company was acquired by alphabet"

In [37]:
for line in word_wrap(rag_chain.invoke(question),100):
  print(line)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SYSTEM:
Answer the question based on your knowledge. use the below context for help
[Document(page_content='\ufeffCompany: Google\nAcquired: Kaggle\nDate(dd-mm-yyyy):
undisclosed\nValue (in billions USD): undisclosed', metadata={'source':
'/content/test_vectordb.csv', 'row': 10}), Document(page_content='\ufeffCompany: Oruka
Therapeutics\nAcquired: ARCA Biopharma\nDate(dd-mm-yyyy): 03-04-2024\nValue (in billions USD):
Undisclosed', metadata={'source': '/content/test_vectordb.csv', 'row': 7}),
Document(page_content='\ufeffCompany: Genmab\nAcquired: ProfoundBio\nDate(dd-mm-yyyy):
03-04-2024\nValue (in billions USD): 1.8', metadata={'source': '/content/test_vectordb.csv', 'row':
6}), Document(page_content='\ufeffCompany: AstraZeneca\nAcquired: Fusion\nDate(dd-mm-yyyy):
19-03-2023\nValue (in billions USD): 2.4', metadata={'source': '/content/test_vectordb.csv', 'row':
4})]
dont provide the context in answer
QUESTION:
which company was acquired by alphabet
ANSWER:
The company that was acquir

In [40]:
for line in word_wrap(llm_chain.invoke({"context":"", "question": question}),100):
  print(line)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SYSTEM:
Answer the question based on your knowledge. use the below context for help
dont provide the context in answer
QUESTION:
which company was acquired by alphabet
ANSWER:
Nest Labs, a maker of smart thermostats and other home devices, has been acquired by Google's parent
company Alphabet for $3.2bn (£2.1bn).
The deal will see Nest become part of the Google [x] unit, which focuses on new technology
initiatives.
Nest's products include the learning thermostat, which uses a mobile app to allow users to remotely
control their heating and cooling systems.
It also makes a smoke detector and a range of connected security cameras.
Alphabet said that Nest would continue to operate as an independent company within its "other bets"
portfolio, which includes companies such as Fiber, Calico and Verily.
Nest's founder and chief executive Tony Hsieh will remain in his position, it added.
In a blog post announcing the acquisition, Mr Page wrote: "We're excited to bring incredible
experiences to m

In [34]:
from flask_app_code import *

In [35]:
main()

Please enter the querywhich company was acquired by alphabet


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ADMIN\\files_folder'